In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import faiss
import sentencepiece

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import Dataset
import pandas as pd
df = pd.read_csv(f"demo.csv", encoding = 'latin-1')
dataset = Dataset.from_pandas(df)

In [ ]:
df

,Line
0,Le mariage est contracté par deux personnes de...
1,"Le mariage d'un Français, même contracté à l'é..."
2,On ne peut contracter un second mariage avant ...
3,Les mineurs ne peuvent contracter mariage sans...
4,Si l'un des deux est mort ou s'il est dans l'i...
...,...
102,Le mariage se dissout : 1° Par la mort de l'u...
103,"Chacun des époux administre, oblige et aliène ..."
104,"Chacun des époux peut se faire ouvrir, sans le..."
105,La femme ne peut contracter un nouveau mariage...


In [ ]:
from transformers import AutoTokenizer, AutoModel
ctx_tokenizer = AutoTokenizer.from_pretrained("etalab-ia/dpr-ctx_encoder-fr_qa-camembert",  do_lower_case=True)
ctx_model = AutoModel.from_pretrained("etalab-ia/dpr-ctx_encoder-fr_qa-camembert", return_dict=True)

Some weights of the model checkpoint at etalab-ia/dpr-ctx_encoder-fr_qa-camembert were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at etalab-ia/dpr-ctx_encoder-fr_qa-camembert and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN t

In [ ]:
dataset

Dataset({
    features: ['Line'],
    num_rows: 136
})

In [ ]:
ds_with_embeddings = dataset.map(lambda example: {'embeddings':ctx_model.forward(ctx_tokenizer(example["Line"], return_tensors='pt')["input_ids"]).pooler_output[0]})

  0%|          | 0/107 [00:00<?, ?ex/s]

In [ ]:
ds_with_embeddings

In [ ]:
ds_with_embeddings.add_faiss_index(column='embeddings')

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Line', 'embeddings'],
    num_rows: 107
})

In [ ]:
from transformers import AutoTokenizer, AutoModel
query = "Les mineurs peuvent-ils contracter un mariage?"
tokenizer = AutoTokenizer.from_pretrained("etalab-ia/dpr-question_encoder-fr_qa-camembert",  do_lower_case=True)
input_ids = tokenizer(query, return_tensors='pt')["input_ids"]
model = AutoModel.from_pretrained("etalab-ia/dpr-question_encoder-fr_qa-camembert", return_dict=True)
embeddings = model.forward(input_ids).pooler_output


Downloading:   0%|          | 0.00/518 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at etalab-ia/dpr-question_encoder-fr_qa-camembert were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at etalab-ia/dpr-question_encoder-fr_qa-camembert and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probab

In [ ]:
embeddings.detach()

tensor([[-2.8417e-03,  1.2934e-01,  8.4247e-02,  4.5188e-03, -2.2053e-01,
         -5.6444e-02, -1.6652e-02,  1.9569e-01,  1.9074e-01,  2.5498e-03,
         -6.7074e-02, -2.0497e-01, -1.5179e-01, -8.6052e-02,  1.4481e-01,
         -2.6925e-02,  2.9066e-02, -1.1452e-01,  8.9970e-02,  1.8932e-01,
         -7.8148e-02, -5.3166e-02, -6.3683e-02, -3.4601e-02, -1.1360e-01,
         -1.3705e-03,  1.7377e-01,  8.3105e-02, -5.1668e-02, -1.2447e-01,
          1.1686e-01, -1.5823e-01,  1.4758e-02,  2.1880e-01,  1.4300e-01,
          1.3911e-01,  6.8843e-02, -2.1642e-02,  1.2255e-01,  1.2356e-01,
         -6.7659e-04,  2.9979e-01, -1.7591e-02,  7.9861e-02,  1.9812e-01,
          5.7270e-02, -5.6486e-02,  5.1636e-02, -5.1172e-02,  1.7226e-01,
         -2.8634e-02,  9.0709e-02, -3.4434e-02, -3.8719e-02,  6.5185e-02,
          7.3008e-02, -1.7190e-01,  9.6396e-03, -5.5451e-02,  6.2572e-02,
         -1.4982e-02,  1.0657e-01,  1.5788e-01, -2.9564e-02, -6.0585e-02,
         -2.5308e-02, -2.7758e-01, -5.

In [ ]:
embeddings2 = embeddings.detach().numpy()


In [ ]:
embeddings2

tensor([[-0.2641, -0.1106,  0.0532, -0.0270,  0.1107,  0.2166, -0.0448,  0.0681,
          0.0582, -0.1399, -0.0714, -0.1111,  0.0461,  0.0496,  0.0022, -0.0980,
          0.1690, -0.1423, -0.1418, -0.0815,  0.1263,  0.1093, -0.1364,  0.3220,
         -0.0955, -0.3239, -0.1099, -0.0405,  0.1419, -0.0650, -0.2713,  0.0939,
          0.1154,  0.0093,  0.1660,  0.0195, -0.0136,  0.0888,  0.1130,  0.0844,
         -0.2061, -0.0670, -0.1025, -0.0991,  0.0385, -0.0221, -0.0808, -0.0091,
         -0.0376, -0.1116, -0.1408, -0.3114, -0.0615, -0.0311, -0.0300,  0.0707,
         -0.1922, -0.1731, -0.0096,  0.0374,  0.1758,  0.0369, -0.0443, -0.0193,
          0.0757,  0.0726,  0.1199,  0.0317,  0.1074, -0.0173,  0.0638,  0.1232,
         -0.1244,  0.0721, -0.1903, -0.0717,  0.2578, -0.0837,  0.0833, -0.0232,
         -0.0572, -0.1113, -0.0813, -0.0252,  0.1905, -0.1345,  0.0540, -0.2107,
          0.1513,  0.0625, -0.2136, -0.2019, -0.2522, -0.0471,  0.0505,  0.1294,
         -0.0866,  0.0314,  

In [ ]:
passages_scores, passages = ds_with_embeddings.get_nearest_examples("embeddings", embeddings2, k=4) # get k nearest neighbors

AttributeError: ignored

In [ ]:
passages['Line']

["Néanmoins, il est loisible au Président de la République de lever, pour des causes graves, les prohibitions portées :  1° Par l'article 161 aux mariages entre alliés en ligne directe lorsque la personne qui a créé l'alliance est décédée ;  2° (Abrogé) ;  3° Par l'article 163.",
 "Lorsque la preuve d'une célébration légale du mariage se trouve acquise par le résultat d'une procédure criminelle, l'inscription du jugement sur les registres de l'état civil assure au mariage, à compter du jour de sa célébration, tous les effets civils, tant à l'égard des époux qu'à l'égard des enfants issus de ce mariage.",
 "Si l'un des époux manque gravement à ses devoirs et met ainsi en péril les intérêts de la famille, le juge aux affaires familiales peut prescrire toutes les mesures urgentes que requièrent ces intérêts.  Il peut notamment interdire à cet époux de faire, sans le consentement de l'autre, des actes de disposition sur ses propres biens ou sur ceux de la communauté, meubles ou immeubles. 

In [ ]:
passages['Line'][0]

"Les dispositions du présent chapitre, en tous les points où elles ne réservent pas l'application des conventions matrimoniales, sont applicables, par le seul effet du mariage, quel que soit le régime matrimonial des époux."

In [ ]:
passage = passages['Line'][0] + '. ' + passages['Line'][1]

In [ ]:
passage = "Il produit aussi ses effets à l'égard des enfants, quand bien même aucun des époux n'aurait été de bonne foi.  Le juge statue sur les modalités de l'exercice de l'autorité parentale comme en matière de divorce.. S'il n'y a ni père, ni mère, ni aïeuls, ni aïeules, ou s'ils se trouvent tous dans l'impossibilité de manifester leur volonté, les mineurs de dix-huit ans ne peuvent contracter mariage sans le consentement du conseil de famille."

In [ ]:
query

'Les mineurs peuvent-ils contracter un mariage ?'

In [ ]:
from transformers import pipeline

nlp = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')


Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210 [00:00<?, ?B/s]

In [ ]:
#for Gradio application
def predict(input, history=[]):
  input_ids = tokenizer(input, return_tensors='pt')["input_ids"]
  embeddings = model.forward(input_ids).pooler_output
  embeddings2 = embeddings.detach().numpy()
  passages_scores, passages = ds_with_embeddings.get_nearest_examples("embeddings", embeddings2, k=4)
  passage = passages['Line'][0] + '. ' + passages['Line'][1]+ '. ' + passages['Line'][2]+ '. ' + passages['Line'][3]
  history.append(input)
  response = nlp({
    'question': query,
    'context': passage
  })['answer']
  history.append(response)
  #response = response.split("<|endoftext|>")
  response = [(history[i], history[i+1]) for i in range(0, len(history)-1, 2)]
  return response, history

In [ ]:
predict('Les mineurs peuvent-ils contracter un mariage?')

[('Les mineurs peuvent-ils contracter un mariage?',
  ' le mineur pourra prêter directement serment en présence des membres du conseil de famille.')]

In [ ]:
import gradio as gr

gr.Interface(fn=predict,
             inputs=["text", "state"],
             outputs=["chatbot", "state"]).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0266e5b7-dafd-411e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://0266e5b7-dafd-411e.gradio.live
